<a href="https://colab.research.google.com/github/rizzken/PreSurgicalNotes_Eval/blob/main/01_data_quality_check.ipynb" target="_parent"><img src="https://colab.research.google.com/assets/colab-badge.svg" alt="Open In Colab"/></a>

In [7]:
import pandas as pd
from google.colab import drive

#drive.mount('/content/drive')
df = pd.read_csv('/content/drive/MyDrive/Colab Notebooks/healthcare_dataset.csv')


Dataset shape (rows, columns): (55500, 15)

Data types:
 Name                   object
Age                     int64
Gender                 object
Blood Type             object
Medical Condition      object
Date of Admission      object
Doctor                 object
Hospital               object
Insurance Provider     object
Billing Amount        float64
Room Number             int64
Admission Type         object
Discharge Date         object
Medication             object
Test Results           object
dtype: object

Missing values per column:
 Name                  0
Age                   0
Gender                0
Blood Type            0
Medical Condition     0
Date of Admission     0
Doctor                0
Hospital              0
Insurance Provider    0
Billing Amount        0
Room Number           0
Admission Type        0
Discharge Date        0
Medication            0
Test Results          0
dtype: int64


In [8]:
# 1. Basic dataset information
print("Dataset shape (rows, columns):", df.shape)
print("\nData types:\n", df.dtypes)
print("\nMissing values per column:\n", df.isnull().sum())

Dataset shape (rows, columns): (55500, 15)

Data types:
 Name                   object
Age                     int64
Gender                 object
Blood Type             object
Medical Condition      object
Date of Admission      object
Doctor                 object
Hospital               object
Insurance Provider     object
Billing Amount        float64
Room Number             int64
Admission Type         object
Discharge Date         object
Medication             object
Test Results           object
dtype: object

Missing values per column:
 Name                  0
Age                   0
Gender                0
Blood Type            0
Medical Condition     0
Date of Admission     0
Doctor                0
Hospital              0
Insurance Provider    0
Billing Amount        0
Room Number           0
Admission Type        0
Discharge Date        0
Medication            0
Test Results          0
dtype: int64


In [9]:
# 2. Check for duplicate rows
print("Number of fully duplicate rows:", df.duplicated().sum())

# If you have a unique identifier column (e.g., 'Patient ID')
if 'Patient ID' in df.columns:
    print("Duplicate Patient IDs:", df.duplicated(subset=['Patient ID']).sum())

Number of fully duplicate rows: 534


In [12]:
# 3. Numerical columns validation (Age, Billing Amount, Room Number)
numerical_cols = ['Age', 'Billing Amount', 'Room Number']

for col in numerical_cols:
    if col in df.columns:
        print(f"\n{col} statistics:\n", df[col].describe())

        # Example: invalid age
        if col == 'Age':
            invalid = df[(df[col] < 0) | (df[col] > 120)]
            print(f"\nInvalid {col} records:\n", invalid[[col, 'Name', 'Gender']])


Age statistics:
 count    55500.000000
mean        51.539459
std         19.602454
min         13.000000
25%         35.000000
50%         52.000000
75%         68.000000
max         89.000000
Name: Age, dtype: float64

Invalid Age records:
 Empty DataFrame
Columns: [Age, Name, Gender]
Index: []

Billing Amount statistics:
 count    55500.000000
mean     25539.316097
std      14211.454431
min      -2008.492140
25%      13241.224652
50%      25538.069376
75%      37820.508436
max      52764.276736
Name: Billing Amount, dtype: float64

Room Number statistics:
 count    55500.000000
mean       301.134829
std        115.243069
min        101.000000
25%        202.000000
50%        302.000000
75%        401.000000
max        500.000000
Name: Room Number, dtype: float64


In [13]:
# 4. Categorical columns inspection
categorical_cols = ['Gender', 'Blood Type', 'Medical Condition', 'Admission Type', 'Medication', 'Test Results']

for col in categorical_cols:
    if col in df.columns:
        print(f"\nValue counts in '{col}' (including NaN):\n", df[col].value_counts(dropna=False))
        print(f"Unique values in '{col}':\n", df[col].unique())


Value counts in 'Gender' (including NaN):
 Gender
Male      27774
Female    27726
Name: count, dtype: int64
Unique values in 'Gender':
 ['Male' 'Female']

Value counts in 'Blood Type' (including NaN):
 Blood Type
A-     6969
A+     6956
AB+    6947
AB-    6945
B+     6945
B-     6944
O+     6917
O-     6877
Name: count, dtype: int64
Unique values in 'Blood Type':
 ['B-' 'A+' 'A-' 'O+' 'AB+' 'AB-' 'B+' 'O-']

Value counts in 'Medical Condition' (including NaN):
 Medical Condition
Arthritis       9308
Diabetes        9304
Hypertension    9245
Obesity         9231
Cancer          9227
Asthma          9185
Name: count, dtype: int64
Unique values in 'Medical Condition':
 ['Cancer' 'Obesity' 'Diabetes' 'Asthma' 'Hypertension' 'Arthritis']

Value counts in 'Admission Type' (including NaN):
 Admission Type
Elective     18655
Urgent       18576
Emergency    18269
Name: count, dtype: int64
Unique values in 'Admission Type':
 ['Urgent' 'Emergency' 'Elective']

Value counts in 'Medication' (inclu